In [1]:
import sys
sys.path.append("Y:\Python\Mega_IT")

from yjk_module.preprocessing import *
from yjk_module.ml_module import *

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *
from hossam.classification import *
from hossam.regression import *

from xgboost import XGBRegressor, XGBClassifier, plot_importance, to_graphviz
from lightgbm import LGBMRegressor, LGBMClassifier, plot_importance
from sklearn.model_selection import RandomizedSearchCV

In [3]:
origin1 = pd.read_excel("./data/pima_indians_diabetes.xlsx")
origin1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# 정제 대상 필드명
zero_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# 0값을 결측치로 대체후 평균으로 다시 치환
df1 = origin1.copy()
df1[zero_features] = df1[zero_features].replace(0, np.nan)
df1_2 = my_replace_missing_value(df1)

# 결과확인
total_count = len(origin1)

for feature in zero_features:
    zero_count = origin1[origin1[feature] == 0][feature].count()
    print(
        "{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %".format(
            feature, zero_count, 100 * zero_count / total_count
        )
    )

print("---" * 30)

for feature in zero_features:
    zero_count = df1_2[df1_2[feature] == 0][feature].count()
    print(
        "{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %".format(
            feature, zero_count, 100 * zero_count / total_count
        )
    )

Glucose 0 건수는 5, 퍼센트는 0.65 %
BloodPressure 0 건수는 35, 퍼센트는 4.56 %
SkinThickness 0 건수는 227, 퍼센트는 29.56 %
Insulin 0 건수는 374, 퍼센트는 48.70 %
BMI 0 건수는 11, 퍼센트는 1.43 %
------------------------------------------------------------------------------------------
Glucose 0 건수는 0, 퍼센트는 0.00 %
BloodPressure 0 건수는 0, 퍼센트는 0.00 %
SkinThickness 0 건수는 0, 퍼센트는 0.00 %
Insulin 0 건수는 0, 퍼센트는 0.00 %
BMI 0 건수는 0, 퍼센트는 0.00 %


In [5]:
x_train, x_test, y_train, y_test = my_train_test_split(df1_2, "Outcome", scalling=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [6]:
support = YjkClassifierSupport()

사용 가능한 모델 : ['LogisticRegression', 'SGDClassifier', 'KNeighborsClassifier', 'LinearSVC', 'SVC', 'GaussianNB', 'DecisionTreeClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'XGBClassifier', 'LGBMClassifier', 'CatboostClassifier']


In [7]:
lgbm = support.call_model('LGBMClassifier')

[LGBMClassifier의 파라미터]
+-------------------+-------------------------------------------------------+--------------------+
| 파라미터명        | 파라미터 노트                                         | 파라미터 범위      |
|-------------------+-------------------------------------------------------+--------------------|
| num_leaves        | 트리당 최대 리프 수                                   | [31, 20, 40]       |
| learning_rate     | 학습률, 높을수록 속도가 빠르지만 과적합 가능성이 높음 | [0.01, 0.05, 0.1]  |
| n_estimators      | 학습을 진행할 부스팅 라운드 수                        | [100, 300, 500]    |
| max_depth         | 트리 깊이                                             | [5, 7, 9, 11, -1]  |
| subsample         | 트리를 구성하는 데이터 샘플링 비율                    | [0.8, 0.9, 1]      |
| colsample_bytree  | 트리를 구성하는 피처의 샘플링 비율                    | [0.6, 0.8, 1]      |
| min_child_samples | 결정 트리의 리프 노드가 되기 위해 필요한 샘플 수      | [10, 20, 30]       |
| min_child_weight  | 트리를 분할하는데 필요한 모든 관측치의 최소 가중치 합 | [0.001, 0.01, 0.1] |
+-------------------+-----

In [8]:
grid = GridSearchCV(lgbm, support.get_params(), n_jobs = -1)

In [9]:
%%time

grid.fit(x_train, y_train)

CPU times: total: 1min 54s
Wall time: 45min 12s


GridSearchCV(estimator=LGBMClassifier(random_state=0, verbose=-1), n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [5, 7, 9, 11, -1],
                         'min_child_samples': [10, 20, 30],
                         'min_child_weight': [0.001, 0.01, 0.1],
                         'n_estimators': [100, 300, 500],
                         'num_leaves': [31, 20, 40],
                         'subsample': [0.8, 0.9, 1]})

- 45분